# Dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

# Data Creation

In [147]:
def generate_sentence(max_len=20, pos=True):
    if pos:
        n = np.random.randint(np.ceil(max_len / 3))
        sentence = n*"a" + n*"b" + n*"c"
        return sentence, len(sentence)
    else:
        n_0 = np.random.randint(0, max_len)
        n_1 = np.random.randint(0, max_len - n_0 + 1)
        n_2 = np.random.randint(0, max_len - n_0 - n_1 + 1)
        sentence = n_0 * "a" + n_1 * "b" + n_2 * "c"
        return sentence, len(sentence)

def create_data(size=10000, balance=0.5):
    data = []
    sentence_lengths = []

    for i in range(int(size*balance)):
        sentence, sentence_length = generate_sentence(pos=True)
        data.append((sentence, 1))
        sentence_lengths.append(sentence_length)
    for i in range(int((size - (size*balance)))):
        sentence, sentence_length = generate_sentence(pos=False)
        data.append((sentence, 0))
        sentence_lengths.append(sentence_length)
    
    random.shuffle(data)
    average_length = sum(sentence_lengths) / len(sentence_lengths)
    
    return data, average_length

data, avg_sent_length = create_data()
print(f"Data Sample:\n{data}")
print(f"Average Sentence Length:\n{avg_sent_length}")


Data Sample:
[('aabbcc', 1), ('aaaaaaaaaaaaaaaaaaa', 0), ('aaaaaaaaaaaaaaaaaacc', 0), ('aaaaaaaaaaaaaacccccc', 0), ('aaaaabbbbbccccc', 1), ('aaaaaaaaaaaaaaaaaabc', 0), ('abc', 1), ('aaaabbbbcccc', 1), ('aaaaaaaaab', 0), ('aaaabbbbcccc', 1), ('aaaabbccc', 0), ('aaaabbbbbbbbbbbbbccc', 0), ('aaaaaabbbbbbcccccc', 1), ('aaabbbbbbbbbbb', 0), ('b', 0), ('aaabbbccc', 1), ('aaaaaaaaaaaaaaaaaabc', 0), ('aaaaaabbbbbbbccccccc', 0), ('aaaaaaaabbbbbbbb', 0), ('aaaaabbbbbccccc', 1), ('bbbbbbbbbbbbbcccccc', 0), ('aaaaaaaaaaaaaaaaabbb', 0), ('aaaaaaaaabbbbbbccc', 0), ('abc', 1), ('aaaaaaaaaaaac', 0), ('aaabbbbbccccccccc', 0), ('aaaabbbbbbbcccccccc', 0), ('aaaaaabbbbbbbbbbbbb', 0), ('aaaaaaaaaabbbbbbbbbb', 0), ('aaaaaabbbbbbcccccc', 1), ('aaaabbbbcccc', 1), ('aabbcc', 1), ('aaaaaaaaaaaaaaaaaabb', 0), ('aaaaaaaaaaaaabbbbcc', 0), ('aaaaaabbbbbbcccccc', 1), ('', 1), ('aaaaabbbbbc', 0), ('aaaaaaaaaaaaaaaaab', 0), ('aaaaabbbbbccccc', 1), ('aaaaaaabbbbbbcccc', 0), ('', 1), ('aaaaaaaaaaaabbbbbbbb', 0), ('abc',

# Part B: Recurrent Neural Networks

In [18]:
# Dependencies
import torch
import torch.nn as nn
import torch.optim as optim

In [207]:
# Encoding data
char_to_index = {'a':ord('a'), 'b':ord('b'), 'c':ord('c')}
index_to_char = {v: k for k, v in char_to_index.items()}
max_l = 20

def creat_tensors():
    X = []
    y = []
    ml = 0

    for sent, label in data:
        X.append([char_to_index[char] for char in sent])
        y.append(label)

    # Padding to be able to convert to tensor
    X = [sent + [0] * (max_l - len(sent)) for sent in X]

    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)

    return X, y

X_train, y_train = creat_tensors()

In [208]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(X_train, y_train)
data_loader = DataLoader(dataset, 32, shuffle=True)

### RNN

In [209]:
# Set device
if torch.cuda.is_available():
    device = 'cuda:0'
elif torch.backends.mps.is_available():
    device = 'mps:0'
else:
    device = 'cpu'
print('GPU State:', device)

GPU State: mps:0


In [212]:
class ElmanRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size) -> None:
        super(ElmanRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.input_size = 1
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.hidden_to_output = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, 32, 128)
        out, _ = self.rnn(x, h0)
        out = self.hidden_to_output(out[:, -1, :])

In [214]:
input_size = 1
hidden_size = 128
num_layers = 1
num_classes = 2
num_epochs = range(20)

model = ElmanRNN(input_size, hidden_size, num_layers, num_classes)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in num_epochs:
    model.train()
    epoch_loss = 0

    for batch_idx, (inputs, targets) in enumerate(data_loader):
        inputs = inputs.unsqueeze(-1)  # Add input_size dimension
        targets = targets
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/100], Loss: {epoch_loss / len(data_loader):.4f}')

AttributeError: 'NoneType' object has no attribute 'size'

### LSTM

In [11]:
pass